# Company brochure generator

Build a company brochure for a company to attract clients, investors and potential recruits.

In [171]:
import os, sys
import requests
import json
import time
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
from pydantic import BaseModel
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

In [97]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

if not openai_api_key:
    print("No OpenAI API key defined")
    sys.exit(1)
else:
    print("OpenAI API key loaded")

OpenAI API key loaded


In [189]:
def normalize_url(base_url, href):
    if href.startswith(('http://', 'https://')):
        return href
    if href.endswith('/'):
        return base_url + href[:-1]
    else:
        return base_url + href

# OpenAI
def get_links(url, selenium=False):
    openai = OpenAI()
    website = Website(url, selenium)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": website.system_prompt},
            {"role": "user", "content": website.user_prompt}
        ],
        response_format = {"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

def get_brochure(json_str, _stream=False):
    openai = OpenAI()
    brochure = Brochure(json_str)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": brochure.system_prompt},
            {"role": "user", "content": brochure.user_prompt}
        ],
        stream = _stream
    )
    if not _stream:
        return response.choices[0].message.content
    else:
        output = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            output += chunk.choices[0].delta.content or ''
            output = output.replace("``", "").replace("markdown", "")
            update_display(Markdown(output), display_id=display_handle.display_id)
        return output

def brochure_maker(url, selenium=False, _stream=True):
    links = get_links(url, selenium)
    brochure = get_brochure(links, _stream)
    if not _stream:
        return brochure

# OSS
class Links(BaseModel):
    links: List[str]

def get_links_oss(url, selenium=False):
    website = Website(url, selenium)
    response = ollama.chat(
        model = "llama3.2",
        messages = [
            {"role": "system", "content": website.system_prompt},
            {"role": "user", "content": website.user_prompt}
        ],
        format = Links.model_json_schema()
    )
    return json.loads(response['message']['content'])

def get_brochure_oss(json_str, _stream=False):
    brochure = Brochure(json_str)
    response = ollama.chat(
        model = "llama3.2",
        messages = [
            {"role": "system", "content": brochure.system_prompt},
            {"role": "user", "content": brochure.user_prompt}
        ],
        stream = _stream
    )
    if not _stream:
        return response['message']['content']
    else:
        output = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in response:
            content = chunk["message"]["content"] or ""
            output += content
            display_text = output.replace("```", "")
            update_display(Markdown(display_text), display_id=display_handle.display_id)
        return output

def brochure_maker_oss(url, selenium=False, _stream=True):
    links = get_links_oss(url, selenium)
    brochure = get_brochure_oss(links, _stream)
    return brochure

In [124]:
class Website:
    url: str
    title: str
    body: str
    text: str
    hrefs: List[str]
    summary: str
    system_prompt: str
    user_prompt: str
    selenium: bool

    def __init__(self, url, selenium=False):
        self.url = url
        self.selenium = selenium
        if selenium:
            self._selenium()
        else:
            self._requests()
        self.system_prompt()
        self.user_prompt()

    def _wait(self, driver, timeout=20):
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.common.by import By
    
        try:
            # Wait for the body to contain something
            WebDriverWait(driver, timeout).until(lambda d: d.find_element(By.TAG_NAME, "body").text.strip() != "")
        except:
            pass

        start = time.time()
        prev_html = ""
        cur_html = ""
        stable_html_count = 0
        # Halt at timeout
        while time.time() - start < timeout:
            cur_html = driver.page_source
            if cur_html == prev_html:
                stable_html_count += 1
                # If the content is 5 times the same then it is considered stable
                if stable_html_count > 5:
                    return
            else:
                stable_html_count = 0
            prev_html = cur_html
            time.sleep(0.5)

    def _requests(self):
        response = requests.get(self.url)
        self.body = response.content
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "Unknown"
        for no_text_tags in soup.body(["script", "style", "img", "input"]):
            no_text_tags.decompose()
        hrefs = soup.find_all('a', href=True)
        self.hrefs = [normalize_url(self.url, href.get('href')) for href in hrefs if not href['href'].startswith(('#', 'javascript:'))]
        self.text = soup.body.get_text(separator="\n", strip=True)

    def _selenium(self):
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--window-size=1920,1080")
        # These will make it look more like a human / normal behavior instead of an automated headless browser
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)

        driver = webdriver.Chrome(options=options)
        driver.get(self.url)
        # Wait until load
        self._wait(driver)
        html = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html, 'html.parser')
        self.title = soup.title.string if soup.title else "Unknown"
        self.body = html
        for no_text_tags in soup.body(["script", "style", "img", "input"]):
            no_text_tags.decompose()
        hrefs = soup.find_all('a', href=True)
        self.hrefs = [normalize_url(self.url, href.get('href')) for href in hrefs if not href['href'].startswith(('#', 'javascript:'))]
        self.text = soup.body.get_text(separator="\n", strip=True)

    def printer(self):
        print(f'Website: {self.url}') 
        print(f'Title: {self.title}')
        print(f'Hrefs: {self.hrefs}')
        print(f'Text Head: {self.text[:100]}...')

    def system_prompt(self):
        self.system_prompt = f"""
    You are provided with a list of Links found on a website. 
    You have to decide which of the links would be most relevant to include in a brochure about the company found at {self.url}. 
    We are interested in pages that may include information of interest such as; About, Company, Career, Jobs, Contact, Social, or anything 
    relevant that can help us build an attractive brochure. 
    Do not include Terms of Service, Privacy Statements, Cookie notices, Payment methods or anything relevant. 
    You always respond in JSON format. 
    You do not output anything else other than the final JSON. 
    Your JSON has to look like the following:
    """
        self.system_prompt += """
    {
        "links": [
            {"type": "base_url", description: "<description>", "url": "<base_url>"},
            {"type": "name", description: "<company_description>", "name": "<company_name>"},
            {"type": "<type>", description: "<description>", "url": "<full_url>"},
            ...
        ]
    }
    """
        self.system_prompt += f"""
    The <base_url> includes only the base URL of the company, you know that already from the link provided above. 
    The <company_description> can be a small sentence up to 10 words describing the company function. 
    The <company_name> has to be the actual company name that you can find out. 
    The <type> can be 1 word for the page type. Eg. About, Company, Career, Jobs, Contact, Social, etc. Do maintain the other static types as provided in the example. 
    The <descrition> can be a small sentence up to 10 words describing the page function. 
    The <url> has to be the actual full url of the page. 
    The `...` mean that you have to do the same for all the relevant links that you picked out and fill the JSON object. 
    You have a limit of up to 30 links. There is no minimum limit.
    """
        
        return self.system_prompt

    def user_prompt(self):
        self.user_prompt = f"""
        The website URL is {self.url}. The website Title is {self.title}. The website Links are provided below: 
        {self.hrefs}
        """
        return self.user_prompt

In [125]:
website = Website('https://gtsig.eu')
website.printer()

Website: https://gtsig.eu
Title: George T.
Hrefs: ['https://www.linkedin.com/in/giorgos-tsigourakos/', 'https://github.com/tsigouris007', 'https://myportal.europisti.gr', 'https://myservices.europisti.gr', 'https://mypolicy.europisti.gr', 'https://physicaltaste.gr', 'https://imikrimaspoli.gr', 'https://rosacolorato.gr', 'https://cavakolokouras.gr', 'https://marmorisfitness.gr/', 'https://alissachnimykonos.com', 'https://alissachniseaside.com', 'https://mdstudio.gr', 'https://diatasi.gr', 'https://politismos.dimosbyrona.gr', 'https://www.wonderfoodland.gr', 'https://beerpack.gr', 'https://nicksvoice.gr', 'https://kostasmaragos.com', 'https://primamateriatherapeutica.gr', 'https://kostasmaragos.com', 'https://gtsig.eu', 'https://www.youtube.com/watch?v=_P0kMzTTL3M', 'https://www.youtube.com/watch?v=A-uMM2zW2io', 'https://www.youtube.com/watch?v=Ew3chxpu9SI', 'https://www.youtube.com/watch?v=jw2DYExMeLM', 'https://www.youtube.com/watch?v=KPL8WJ_MH-I', 'https://www.youtube.com/watch?v=a73G

In [126]:
website = Website('https://airbnb.com')
website.printer()

Website: https://airbnb.com
Title: Airbnb: Vacation Rentals, Cabins, Beach Houses, Unique Homes & Experiences
Hrefs: ['https://airbnb.com', 'https://airbnb.com/homes', 'https://airbnb.com/experiences', 'https://airbnb.com/services', 'https://airbnb.com/help', 'https://airbnb.com/co-hosts/home', 'https://airbnb.com/giftcards', 'https://airbnb.com/signup_login', 'https://airbnb.com/united-states/stays/cabins', 'https://airbnb.com/united-states/stays/treehouses', 'https://airbnb.com/united-states/stays/glamping', 'https://airbnb.com/united-states/stays/tiny-houses', 'https://airbnb.com/united-states/stays/beach-houses', 'https://airbnb.com/united-states/stays/campers', 'https://airbnb.com/united-states/stays/lakehouses', 'https://airbnb.com/united-states/stays/yurts', 'https://airbnb.com/united-kingdom/stays/yurts', 'https://airbnb.com/united-states/stays/castles', 'https://airbnb.com/united-states/stays/houseboats', 'https://airbnb.com/united-kingdom/stays/holiday-caravans', 'https://air

In [110]:
website = Website('https://airbnb.com', selenium=True)
website.printer()

Website: https://airbnb.com
Title: Airbnb | Vacation rentals, cabins, beach houses, & more
Hrefs: ['https://airbnb.com', 'https://airbnb.com/homes', 'https://airbnb.com/experiences', 'https://airbnb.com/services', 'https://airbnb.com/s/Athens/homes?place_id=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=FLEXIBLE_DATES&search_type=HOMEPAGE_CAROUSEL_CLICK', 'https://airbnb.com/rooms/1323300909826473606?check_in=2025-11-07&check_out=2025-11-09&photo_id=2191451835&source_impression_id=p3_1751363496_P3XCNV6l1a4B2VJq&previous_page_section_name=1000', 'https://airbnb.com/rooms/1328795490105870915?check_in=2025-10-31&check_out=2025-11-02&photo_id=2063189277&source_impression_id=p3_1751363496_P3nfSKD81SrIs6ZQ&previous_page_section_name=1000', 'https://airbnb.com/rooms/1323072299132299735?check_in=2025-11-07&check_out=2025-11-09&photo_id=2081625969&source_impression_id=p3_1751363496_P3oME6u6KwBzGfyq&previous_page_section_name

In [144]:
links1 = get_links("https://gtsig.eu")
links1

{'links': [{'type': 'base_url',
   'description': 'Company website',
   'url': 'https://gtsig.eu'},
  {'type': 'name',
   'description': "George T.'s professional page",
   'name': 'George T.'},
  {'type': 'Social',
   'description': 'LinkedIn profile of Giorgos Tsigourakos',
   'url': 'https://www.linkedin.com/in/giorgos-tsigourakos/'},
  {'type': 'Social',
   'description': 'GitHub profile of Giorgos Tsigourakos',
   'url': 'https://github.com/tsigouris007'}]}

In [133]:
links2 = get_links("https://airbnb.com")
links2

{'links': [{'type': 'base_url',
   'description': 'The main website of Airbnb',
   'url': 'https://airbnb.com'},
  {'type': 'name',
   'description': 'Connects travelers with unique accommodations',
   'name': 'Airbnb'},
  {'type': 'About',
   'description': 'Learn about Airbnb and its mission',
   'url': 'https://airbnb.com/press/news'},
  {'type': 'Career',
   'description': 'Browse job opportunities at Airbnb',
   'url': 'https://airbnb.com/careers'},
  {'type': 'Contact',
   'description': 'Get support and assistance from Airbnb',
   'url': 'https://airbnb.com/help'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Facebook',
   'url': 'https://www.facebook.com/airbnb'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Twitter',
   'url': 'https://twitter.com/airbnb'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Instagram',
   'url': 'https://instagram.com/airbnb'},
  {'type': 'Community',
   'description': 'Join the Airbnb community for support',
   '

In [132]:
links3 = get_links("https://airbnb.com", selenium=True)
links3

{'links': [{'type': 'base_url',
   'description': 'The main website for Airbnb',
   'url': 'https://airbnb.com'},
  {'type': 'name',
   'description': 'Online marketplace for lodging and travel',
   'name': 'Airbnb'},
  {'type': 'About',
   'description': 'Learn more about Airbnb',
   'url': 'https://airbnb.com/about'},
  {'type': 'Careers',
   'description': 'View job opportunities at Airbnb',
   'url': 'https://airbnb.com/careers'},
  {'type': 'Contact',
   'description': 'Get in touch with Airbnb',
   'url': 'https://airbnb.com/help/home?from=footer'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Facebook',
   'url': 'https://www.facebook.com/airbnb'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Twitter',
   'url': 'https://twitter.com/airbnb'},
  {'type': 'Social',
   'description': 'Follow Airbnb on Instagram',
   'url': 'https://instagram.com/airbnb'},
  {'type': 'Press',
   'description': "Read Airbnb's latest news",
   'url': 'https://airbnb.com/press/n

In [180]:
class Brochure:
    json: str

    def __init__(self, json):
        self.json = json
        self.system_prompt()
        self.user_prompt()

    def printer(self):
        print(f'JSON: {self.json}')

    def system_prompt(self):
        self.system_prompt = f"""
    You are a brochure builder. 
    You are provided with a JSON structured object that includes company details. 
    You have to build a short brochure showcasing the company to attract clients, recruits and stakeholders. 
    The output has to be formatted in Markdown. 
    Do not output thoughts or anything else other than the final Markdown brochure.
    """
        
        return self.system_prompt

    def user_prompt(self):
        self.user_prompt = f"""
    The company details follow: 
    {self.json}
    """
        return self.user_prompt

In [151]:
brochure = get_brochure(links1)
display(Markdown(brochure))

---
### Welcome to GTSIG!

**Your Gateway to Innovative Solutions**

At GTSIG, we specialize in delivering cutting-edge solutions tailored for the modern world. Whether you are looking for exceptional service or a partnership that drives innovation, our team is here to elevate your business.

#### About Us:
GTSIG is committed to leveraging technology for impactful results. Our expertise spans various domains, ensuring we meet the unique needs of each client. With a focus on integrity, collaboration, and excellence, we pride ourselves in fostering meaningful relationships with clients, recruits, and stakeholders alike.

#### Connect with Us
Explore more about our projects and join our community:

- **Website:** [gtsig.eu](https://gtsig.eu)
- **George T.**: Visit [George T.'s professional page](#) to learn more about our leadership and expertise.
- **LinkedIn:** Connect with our founder on [LinkedIn](https://www.linkedin.com/in/giorgos-tsigourakos/) for insights and updates.
- **GitHub:** Check out our code and contributions on [GitHub](https://github.com/tsigouris007).

#### Join Us
Are you ready to take the next step in your career? Discover exciting opportunities with GTSIG. We are always looking for passionate individuals who want to make a difference.

#### Partner with Us
Looking for a reliable partner to grow your business? GTSIG offers tailored solutions to help you achieve your goals. Let's build the future together!

---

For inquiries, reach out to us through our website or connect with George T. on LinkedIn. We look forward to collaborating with you!

In [149]:
brochure = get_brochure(links3)
display(Markdown(brochure))

---

**Welcome to Airbnb!**  
*Your Gateway to Unique Lodging and Travel Experiences*

---

**About Us**  
Airbnb is a leading online marketplace specializing in short-term lodging and travel experiences around the globe. We connect travelers with hosts offering a wide range of accommodations — from cozy apartments to luxurious villas.

[Learn more about us](https://airbnb.com/about)

---

**Join Our Team**  
Are you looking to make an impact in the travel industry? Airbnb offers a plethora of career opportunities for talented individuals who are passionate about travel and hospitality.

[Explore Career Opportunities](https://airbnb.com/careers)

---

**Stay In Touch**  
Have questions? Our team is here to assist you! Feel free to reach out to us anytime.

[Contact Us](https://airbnb.com/help/home?from=footer)

---

**Connect with Us**  
Stay updated with the latest from Airbnb. Follow us on our social media platforms:  
- [Facebook](https://www.facebook.com/airbnb)  
- [Twitter](https://twitter.com/airbnb)  
- [Instagram](https://instagram.com/airbnb)  

---

**For the Latest News**  
Curious about what's happening at Airbnb? Check out our press releases for the latest updates and insights.

[Airbnb Press News](https://airbnb.com/press/news)

---

**Investors**  
Interested in learning more about our company from an investment perspective? Dive into our resources designed for current and prospective investors.

[Investor Information](https://investors.airbnb.com)

---

**Visit Us**  
Explore what Airbnb has to offer at our official website:

[Visit Airbnb](https://airbnb.com)

---

Join us at Airbnb as we redefine travel and hospitality, creating unforgettable experiences for millions around the world! 

---

In [159]:
brochure = get_brochure(links1, _stream=True)

---

**Welcome to GTSIG: The Future of Innovation**

At GTSIG, we are dedicated to pioneering innovative solutions that drive progress and foster growth in various industries. Whether you're a potential client seeking cutting-edge services, a talented professional looking to join a dynamic team, or a stakeholder interested in collaboration opportunities, GTSIG is the place for you.

**Our Vision**

GTSIG strives for excellence in delivering tailored solutions that meet the diverse needs of our clients. We believe in the power of teamwork, creativity, and technology to transform ideas into reality.

**Explore Our Services**

From consulting to technology solutions, we provide a comprehensive range of services designed to propel your business forward. Our team of experts is here to guide you through every step of your project, ensuring optimal results and satisfaction.

**Join Our Team**

Looking to elevate your career? At GTSIG, we prioritize a culture of innovation, growth, and collaboration. We are on the lookout for talented individuals eager to make an impact in their field. Being a part of our team means contributing to meaningful projects while being supported in your professional development.

**Connect with Us**

Stay connected and get to know us better:

- Visit our website: [GTSIG Official Website](https://gtsig.eu)
- Connect with George T.: [George T.'s Professional Page](https://gtsig.eu)
- Follow Giorgos Tsigourakos on LinkedIn: [LinkedIn Profile](https://www.linkedin.com/in/giorgos-tsigourakos/)
- Check out our projects on GitHub: [GitHub Profile](https://github.com/tsigouris007)

**Get in Touch!**

Discover how GTSIG can empower your organization today. We look forward to collaborating with you!

---

**GTSIG - Innovate. Transform. Succeed.**

In [160]:
brochure = get_brochure(links2, _stream=True)

# Discover Airbnb

Welcome to **Airbnb**, where we connect travelers with unique accommodations around the globe. Explore a world of possibilities and make unforgettable memories with us!

## 🌍 About Us
At Airbnb, our mission is to create a world where anyone can belong anywhere. We offer travelers an array of unique lodging options, from cozy cabins to luxurious villas, making every journey memorable.

**Learn more about our mission:** [About Airbnb](https://airbnb.com/press/news)

## 💼 Career Opportunities
Join our dynamic team and be part of something great! We are constantly searching for passionate individuals to help us redefine travel.

**Browse job opportunities:** [Career at Airbnb](https://airbnb.com/careers)

## 📞 Contact Us
Need assistance? Our dedicated support team is here to help you with any questions or concerns.

**Get support:** [Contact Airbnb](https://airbnb.com/help)

## 🌐 Connect with Us
Stay updated and join our vibrant community! Follow us on our social media platforms:

- [Facebook](https://www.facebook.com/airbnb)
- [Twitter](https://twitter.com/airbnb)
- [Instagram](https://instagram.com/airbnb)

## 🤝 Join the Community
Enhance your experience by connecting with like-minded individuals. Our community platform offers support and insights from fellow travelers and hosts.

**Join the Airbnb Community:** [Community Center](https://community.withairbnb.com/t5/Community-Center/ct-p/community-center)

---

**Discover your next adventure with Airbnb.** Visit our website for more information: [Visit Airbnb](https://airbnb.com)

In [194]:
brochure_maker("https://www.skroutz.gr")

# Welcome to Skroutz

## Your Online Marketplace for Products

Discover an endless variety of products at competitive prices with Skroutz, your go-to online marketplace where convenience meets quality.

### Why Choose Skroutz?

- **Wide Selection**: Explore millions of products across various categories, ensuring you find exactly what you need.
- **Best Prices**: Compare prices from different sellers to get the best deals.
- **User-Friendly Experience**: Enjoy a seamless browsing and purchasing process tailored to your needs.
- **Customer Reviews**: Make informed decisions based on real customer reviews and ratings.

### Join Us!

At Skroutz, we believe in innovation and customer satisfaction. Be part of our journey as we redefine online shopping!

### Explore More

Learn more about us and start shopping today!  
[Visit our website](https://www.skroutz.gr)

---

Thank you for considering Skroutz! Together, let's shape the future of online shopping.

In [173]:
links4 = get_links_oss("https://gtsig.eu")
links4

{'links': ['{"type": "base_url", "description": "Company website", "url": "https://gtsig.eu"}',
  '{"type": "name", "description": "George T.. company name", "name": "GTSIG"}',
  '{"type": "About", "description": "Information about the company", "url": "https://gtsig.eu/about"}',
  '{"type": "Company", "description": "General information about GTSIG", "url": "https://gtsig.eu/company"}',
  '{"type": "Career", "description": "Information on job opportunities at GTSIG", "url": "https://gtsig.eu/career"}',
  '{"type": "Jobs", "description": "List of available jobs at GTSIG", "url": "https://gtsig.eu/jobs"}',
  '{"type": "Contact", "description": "Way to contact the company", "url": "https://gtsig.eu/contact"}',
  '{"type": "Social", "description": "Links to social media profiles of GTSIG", "url": "https://gtsig.eu/social"}',
  '{"type": "Physical Taste", "description": "Information about food and beverage services", "url": "https://physicaltaste.gr"}',
  '{"type": "Imikrimas Poli", "descr

In [183]:
brochure = get_brochure_oss(links4)
display(Markdown(brochure))

# About GTSIG
================

At GTSIG, we are dedicated to providing innovative solutions for a better tomorrow. Our company is built on the principles of quality, integrity, and excellence.

## Our Services
---------------

We offer a wide range of services that cater to different needs and preferences. Below, you will find an overview of our services:

### **Food and Beverage**

*   [Wonder Food Land](https://www.wonderfoodland.gr): Experience the art of food and wine at its finest.
*   [Beer Pack](https://beerpack.gr): Discover a world of craft beers and delicious snacks.

### **Wellness and Health**

*   [Imikrimas Poli](https://imikrimaspoli.gr)
*   [Primamateria Therapeutica](https://primamateriatherapeutica.gr)

### **Beauty and Wellness**

*   [Rosacolorato](https://rosacolorato.gr)
*   [Cavakolokouras](https://cavakolokouras.gr)

### **Fitness and Sports**

*   [Marmoris Fitness](https://marmorisfitness.gr/)

### **Accommodation and Tourism**

*   [Alissa by Niki Mykonos](https://alissachnimykonos.com)
*   [Alissa by Niki Side](https://alissachniseaside.com)

### **Design and Consulting**

*   [MD Studio](https://mdstudio.gr)
*   [Diatasi](https://diatasi.gr)

## About Us
----------

Our company is built on the values of innovation, quality, and customer satisfaction. Our founder, Giorgos Tsigourakos, has a strong vision for creating a better future through technology and design.

### **Meet our Team**

*   [George T.. LinkedIn profile](https://www.linkedin.com/in/giorgos-tsigourakos/): Learn more about our founder's journey.
*   [tsigouris007 GitHub profile](https://github.com/tsigouris007): Explore our team's projects and contributions.

## Get in Touch
----------------

Ready to learn more or get started? We are here to help. Please feel free to reach out to us at:

[Company Website](https://gtsig.eu)

### **Stay Connected**

Follow us on social media for the latest updates, news, and promotions:

*   [LinkedIn](https://www.linkedin.com/company/gtsig/)
*   [GitHub](https://github.com/tsigouris007)
*   [Twitter](https://twitter.com/gtsig)

In [190]:
brochure = get_brochure_oss(links4, _stream=True)

# Welcome to GTSIG
=====================================

[Company Website](https://gtsig.eu)

GTSIG is a company dedicated to providing innovative solutions in various sectors. We are committed to excellence and customer satisfaction.

## About Us
-------------

Learn more about our history, mission, and values at [About Us](https://gtsig.eu/about).

## Services
----------

We offer a wide range of services including:

*   [Imikrimas Poli](https://imikrimaspoli.gr) - Wellness Services
*   [Rosacolorato](https://rosacolorato.gr) - Beauty Services
*   [Cavakolokouras](https://cavakolokouras.gr) - Wellness Services
*   [Marmoris Fitness](https://marmorisfitness.gr/) - Fitness Services
*   [Alissa by Niki Mykonos](https://alissachnimykonos.com) - Accommodation Services
*   [Alissa by Niki Side](https://alissachniseaside.com) - Accommodation Services
*   [MD Studio](https://mdstudio.gr) - Design Services
*   [Diatasi](https://diatasi.gr) - Consulting Services
*   [Politismos](https://politismos.dimosbyrona.gr) - Politics and Governance
*   [Wonder Food Land](https://www.wonderfoodland.gr) - Food Services
*   [Beer Pack](https://beerpack.gr) - Beverage Services
*   [Nick's Voice](https://nicksvoice.gr) - Music and Audio Services
*   [Kostas Maragos](https://kostasmaragos.com) - Consulting Services
*   [Primamateria Therapeutica](https://primamateriatherapeutica.gr) - Healthcare Services

## Contact Us
--------------

Get in touch with us at [Contact Us](https://gtsig.eu/contact).

## Social Media
---------------

Follow us on social media at:

[Company LinkedIn Profile](https://www.linkedin.com/in/giorgos-tsigourakos/)
[Company GitHub Profile](https://github.com/tsigouris007)
[Social Media Profiles](https://gtsig.eu/social)

## Physical Taste
-----------------

Learn more about our food and beverage services at [Physical Taste](https://physicaltaste.gr).

## Imikrimas Poli
----------------

Discover the benefits of our wellness services at [Imikrimas Poli](https://imikrimaspoli.gr).

In [193]:
brochure_maker_oss("https://airbnb.com", selenium=True)

# Welcome to Our Company
=====================================

We are a leading provider of unique and cozy accommodations, offering stunning experiences in the United States, United Kingdom, and Australia.

**Our Story**
---------------

At [Company Name], we believe that everyone deserves a comfortable and memorable stay. We curate a selection of the most impressive cabins, yurts, and other unique abodes, all handpicked to provide an unforgettable experience for our guests.

**Discover Our Accommodations**
------------------------------

*   **United States: Stays in Cabins** - Explore the best of America's great outdoors with our curated collection of cozy cabins. From the mountains to the coastlines, we've got you covered.
    [Learn More](https://airbnb.com/united-states/stays/cabins)
*   **United Kingdom: Stays in Yurts** - Experience the magic of the British countryside with our selection of rustic yurts. Perfect for nature lovers and adventure seekers alike.
    [Discover Now](https://airbnb.com/united-kingdom/stays/yurts)
*   **Australia: Stays in Cabins** - Down under, we offer a unique blend of tropical getaways and rugged wilderness adventures in our beautifully crafted cabins.
    [Explore Our Options](https://airbnb.com/australia/stays/cabins)

# Join the Adventure
---------------------

Whether you're planning a family vacation, a romantic getaway, or an adventure with friends, we invite you to experience the best of travel with us. Book your stay today and create unforgettable memories!

"# Welcome to Our Company\n=====================================\n\nWe are a leading provider of unique and cozy accommodations, offering stunning experiences in the United States, United Kingdom, and Australia.\n\n**Our Story**\n---------------\n\nAt [Company Name], we believe that everyone deserves a comfortable and memorable stay. We curate a selection of the most impressive cabins, yurts, and other unique abodes, all handpicked to provide an unforgettable experience for our guests.\n\n**Discover Our Accommodations**\n------------------------------\n\n*   **United States: Stays in Cabins** - Explore the best of America's great outdoors with our curated collection of cozy cabins. From the mountains to the coastlines, we've got you covered.\n    [Learn More](https://airbnb.com/united-states/stays/cabins)\n*   **United Kingdom: Stays in Yurts** - Experience the magic of the British countryside with our selection of rustic yurts. Perfect for nature lovers and adventure seekers alike.\n 

In [192]:
brochure_maker_oss("https://www.skroutz.gr")

# Welcome to Skroutz Online Travel Agency
=====================================

[Company Website](https://www.skroutz.gr)

Skroutz is an online travel agency that offers a wide range of travel services, including flights, hotels, packages, car rentals, and more. Our mission is to provide our customers with the best possible travel experiences at competitive prices.

### About Us
--------

[Learn about Skroutz and our mission](https://www.skroutz.gr/en/about/)

At Skroutz, we are committed to providing exceptional customer service and building long-lasting relationships with our clients. Our team of experts is dedicated to helping you plan your dream vacation.

### What We Offer
---------------

* [Flights](https://www.skroutz.gr/flights/)
* [Hotels](https://www.skroutz.gr/hotels/)
* [Packages](https://www.skroutz.gr/packages/)
* [Car Rental](https://www.skroutz.gr/car-rental/) (Coming soon!)
* [Cruise Holidays](https://www.skroutz.gr/cruise-holidays/) (Coming soon!)
* [Travel Insurance](https://www.skroutz.gr/travel-insurance/) (Coming soon!)
* [Travel Guides](https://www.skroutz.gr/travel-guides/) (Coming soon!)

### How to Contact Us
--------------------

* [Contact Form](https://www.skroutz.gr/contact-form/)
* [Email](mailto:info@skroutz.gr)
* [Phone](+30 210 333 6666)

### Stay in Touch
---------------

Follow us on social media:

[Facebook](https://www.facebook.com/skroutzgr) | [Twitter](https://twitter.com/SkroutzGR) | [Instagram](https://www.instagram.com/skroutz.gr)

# Join Our Community
---------------------

Stay up-to-date with the latest news and offers from Skroutz. Subscribe to our newsletter today!

[Subscribe Now](https://www.skroutz.gr/newsletter/)

'# Welcome to Skroutz Online Travel Agency\n=====================================\n\n[Company Website](https://www.skroutz.gr)\n\nSkroutz is an online travel agency that offers a wide range of travel services, including flights, hotels, packages, car rentals, and more. Our mission is to provide our customers with the best possible travel experiences at competitive prices.\n\n### About Us\n--------\n\n[Learn about Skroutz and our mission](https://www.skroutz.gr/en/about/)\n\nAt Skroutz, we are committed to providing exceptional customer service and building long-lasting relationships with our clients. Our team of experts is dedicated to helping you plan your dream vacation.\n\n### What We Offer\n---------------\n\n* [Flights](https://www.skroutz.gr/flights/)\n* [Hotels](https://www.skroutz.gr/hotels/)\n* [Packages](https://www.skroutz.gr/packages/)\n* [Car Rental](https://www.skroutz.gr/car-rental/) (Coming soon!)\n* [Cruise Holidays](https://www.skroutz.gr/cruise-holidays/) (Coming soo